[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/floleuerer/ml-tutorials/blob/main/01_fastai_pets.ipynb)

# Vorbereitungen

Python-Pakete von [fast.ai](https://www.fast.ai) installieren und importieren.

In [ ]:
!pip install --upgrade fastai fastcore

In [ ]:
from fastai.vision.all import *

# Daten laden

Fast.ai liefert verschiedene Standard-Datensets mit, die häufig für DeepLearning-Benchmarks verwendet werden. Das gebräuchlichste ist das PETS-Dataset. Es besteht aus 37 verschiedenen Hunde- und Katzenrassen und umfasst insgesamt ca. 7400 Bilder.

Datensatz herunterladen und den Pfad als `path`-Variable speichern. Anschlißend die Bilderpfade als `files`-Liste laden.

In [ ]:
path = untar_data(URLs.PETS)

In [ ]:
(path/'images').ls()

In [ ]:
files = get_image_files(path/"images")
len(files)

Jupyter Notebooks unterstützt die Anzeige von Bildern. Dazu  müssen wir mit `PILImage.create` ein Bild-Objekt (Variable `img`) und zeigen es an. Um ein anderes Bild anzuzeigen, kannst du die Varible `img_number` ändern und die Zelle erneut ausführen.

In [ ]:
img_number = 7
img = PILImage.create(files[img_number])
img

# Cats-vs-Dogs

Als einfachstes Modell trainieren wir einen Cats-vs-Dogs Classifier. D.h. das Modell wird trainiert um zwischen Hunden und Katzen zu unterscheiden. Dazu machen wir uns zur Nutze, dass die Katzenbilder mit einen Großbuchstaben beginnen (Egyptian_Mau_167.jpg) und die Hundebilder mit kleinbuchstaben (shiba_inu_136.jpg).

In [ ]:
def label_func(f): return 'Cat' if f[0].isupper() else 'Dog'

In [ ]:
dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

# Ergebnisse anzeigen und interpretieren

`learn.show_results()` zeigt Beispielbilder und deren Klassifizierung an (rot bedeutet, dass das Modell das Bild falsch klassifiziert hat).

In [ ]:
learn.show_results()

Um genauer zu untersuchen wo das Modell Fehler macht, kann man sich die Bilder mit dem höchsten `loss` anzeigen lassen. Ein hohes `loss` Bedeutet, dass ein Bild entweder Falsch klassifiziert wurde, oder das Modell sich bei der richtigen Klassifizierung unsicher war. Bei den folgenden Bildern hatte das Modell also Schwierigkeiten eine korrekte Entscheidung zu treffen.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(15,10))

Mit der `Confusion Matrix` sieht man, wieviele Bilder "verwechselt" also falsch Klassifiziert wurden.

In [ ]:
interp.plot_confusion_matrix()